# More Data Preprocessing (with Pandas)

Pandas is a very useful data analytics package within Python.

Let's start with some random data.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.DataFrame(np.random.randn(5, 3), index=['a', 'c', 'e', 'f', 'h'], columns=['one', 'two', 'three'])
df

,one,two,three
a,-0.411736,-1.135656,-0.614899
c,-0.578368,0.769940,-1.260277
e,-1.624875,-0.024840,0.549979
f,2.225163,0.000492,0.232036
h,1.672547,0.114418,2.138612


We know with Pandas we can mix & match data types, so let us add two more columns, named 'four' and 'five':

In [3]:
df['four'] = 'bar'
df['five'] = df['one'] > 0
df

,one,two,three,four,five
a,-0.411736,-1.135656,-0.614899,bar,False
c,-0.578368,0.769940,-1.260277,bar,False
e,-1.624875,-0.024840,0.549979,bar,False
f,2.225163,0.000492,0.232036,bar,True
h,1.672547,0.114418,2.138612,bar,True


Adding new rows is also simple. Below we include three extra empty rows:

In [4]:
df2 = df.reindex(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h'])
df2

,one,two,three,four,five
a,-0.411736,-1.135656,-0.614899,bar,False
b,NaN,NaN,NaN,NaN,NaN
c,-0.578368,0.769940,-1.260277,bar,False
d,NaN,NaN,NaN,NaN,NaN
e,-1.624875,-0.024840,0.549979,bar,False
f,2.225163,0.000492,0.232036,bar,True
g,NaN,NaN,NaN,NaN,NaN
h,1.672547,0.114418,2.138612,bar,True


# Your Turn Here

Do you still remember how to index a row/column?

In [5]:
#### index row 'c' below
df2.loc['c']

one     -0.578368
two       0.76994
three    -1.26028
four          bar
five        False
Name: c, dtype: object

In [6]:
#### index column 'two' below
df2.loc[:,'two']

a   -1.135656
b         NaN
c    0.769940
d         NaN
e   -0.024840
f    0.000492
g         NaN
h    0.114418
Name: two, dtype: float64

pandas has two functions isnull() and notnull() that return boolean objects when called.

In [7]:
pd.isnull(df2['one'])

a    False
b     True
c    False
d     True
e    False
f    False
g     True
h    False
Name: one, dtype: bool

In [8]:
pd.notnull(df2['one'])

a     True
b    False
c     True
d    False
e     True
f     True
g    False
h     True
Name: one, dtype: bool

Missing values propagate naturally through arithmetic operations between pandas objects.

In [9]:
a = df[['one','two']]
a['one']['a':'e'] = float('nan')
a

/opt/tljh/user/lib/python3.6/site-packages/pandas/core/series.py:1240: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_with(key, value)
/opt/tljh/user/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3319: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


,one,two
a,NaN,-1.135656
c,NaN,0.769940
e,NaN,-0.024840
f,2.225163,0.000492
h,1.672547,0.114418


In [10]:
b = df[['one','two','three']]
b

,one,two,three
a,-0.411736,-1.135656,-0.614899
c,-0.578368,0.769940,-1.260277
e,-1.624875,-0.024840,0.549979
f,2.225163,0.000492,0.232036
h,1.672547,0.114418,2.138612


In [11]:
a + b

,one,three,two
a,NaN,NaN,-2.271311
c,NaN,NaN,1.539881
e,NaN,NaN,-0.049680
f,4.450326,NaN,0.000984
h,3.345095,NaN,0.228836


# How to deal with Missing Values

## Deleting Missing Values

The simplest method is always dropping all missing values - but it is highly **discouraged!!!**

In [12]:
a['one'].dropna()

f    2.225163
h    1.672547
Name: one, dtype: float64

By default, dropna() will drop any row containing **NaN** values, but you can change that by using the *axis=* and *thresh=* arguments.

**NOTE**: Dropping rows or columns have different uses.

In [13]:
#### This statement drop any column with NaN values
a.dropna(axis=1)

,two
a,-1.135656
c,0.769940
e,-0.024840
f,0.000492
h,0.114418


In [14]:
#### thresh determines how many non-NaN values a column/row should have without being dropped
c = a + b
c.dropna(axis=1, thresh=2)

,one,two
a,NaN,-2.271311
c,NaN,1.539881
e,NaN,-0.049680
f,4.450326,0.000984
h,3.345095,0.228836


You can also use the *how=* argument to determine how do you want to remove the NaN values.

In [15]:
#### By default, dropna() drops column/row with any NaN values
#### how = 'all' changes that to dropping column/row with all NaN values
c.dropna(axis=1, how='all')

,one,two
a,NaN,-2.271311
c,NaN,1.539881
e,NaN,-0.049680
f,4.450326,0.000984
h,3.345095,0.228836


You can refer to the [pandas.dropna() docs](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.dropna.html) for more information.

## Imputing Missing Values

Imputing means filling missing values - you can do that when the missing and non-missing values are in some type of relationship.

In [16]:
my_series = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
my_series

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [17]:
#### you can fill missing values with a specific value (0)
my_series.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

Alternatively, we can specify a forward-fill to propagate the previous value forward:

In [18]:
# forward-fill
my_series.fillna(method='ffill')

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

Or we can specify a back-fill to propagate the next values backward:

In [19]:
my_series.fillna(method='bfill')

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

## Your Turn Here
Aforementioned method can be applied to dataframes. 

In [32]:
#### Let us generate a random dataframe
rand_df = pd.DataFrame(np.random.randn(5, 3), 
                  index=['a', 'b', 'c', 'd', 'e'],
                  columns=['one', 'two', 'three'])
rand_df = rand_df.mask(np.random.random(rand_df.shape) < .3)
rand_df

,one,two,three
a,-0.473427,NaN,NaN
b,NaN,NaN,NaN
c,NaN,-0.244642,NaN
d,0.874850,NaN,-1.053710
e,-0.420117,-1.383031,-0.122622


Your tasks are as follows:

- fill missing values in column **one** with value 1;
- fill missing values in column **two** with forward-filling;
- fill missing values in column **three** with backward-filling.

In [33]:
#### insert your code here
# fillna column 'one'
rand_df['one'] = rand_df['one'].fillna(1)
# fillna column 'two'
rand_df['two'] = rand_df['two'].fillna(method='ffill')
# fillna column 'three'
rand_df['three'] = rand_df['three'].fillna(method='bfill')
rand_df

,one,two,three
a,-0.473427,NaN,-1.053710
b,1.000000,NaN,-1.053710
c,1.000000,-0.244642,-1.053710
d,0.874850,-0.244642,-1.053710
e,-0.420117,-1.383031,-0.122622


A useful approach for imputing your missing data is to use mean/mode to replace missing data - the reason behind this logic is that if we are going to *guess* the values of the missing data, the highest chance would be guessing it to be the mean/mode if the data follows **normal** distribution.

In [34]:
#### let us generate another DF
my_df = pd.DataFrame(np.random.randn(5, 3), 
                  index=['a', 'b', 'c', 'd', 'e'],
                  columns=['A', 'B', 'C'])
my_df = my_df.mask(np.random.random(my_df.shape) < .3)
my_df

,A,B,C
a,0.110889,-0.475364,-1.020744
b,-0.679679,-0.291903,NaN
c,0.138735,NaN,-0.241587
d,NaN,NaN,NaN
e,NaN,-0.478081,-2.025566


In [35]:
#### Let us check if there is any missing value in the df
my_df.isnull().values.any()

True

In [36]:
#### Then we are getting the mean of the DF
#### Note that since we only 
my_df.mean()

A   -0.143352
B   -0.415116
C   -1.095966
dtype: float64

In [37]:
df_filled = my_df.fillna(my_df.mean())
df_filled

,A,B,C
a,0.110889,-0.475364,-1.020744
b,-0.679679,-0.291903,-1.095966
c,0.138735,-0.415116,-0.241587
d,-0.143352,-0.415116,-1.095966
e,-0.143352,-0.478081,-2.025566


In [38]:
#### Now let us check again if there is any missing values
df_filled.isnull().values.any()

False

More info regarding how to handle missing data can be found [here](https://machinelearningmastery.com/handle-missing-data-python/).

# Other Tasks in Data Preprocessing

- Handling categorical data (coding)
- Handling imbalanced data
- feature engineering

These topics will be covered in later part of this class.